# Playground
The function of this file is to be able to quickly test things.

In [1]:
import numpy as np
import matplotlib as plt
import json 
import tensorflow as tf

2024-11-11 16:13:37.460892: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Run a test model here

In [2]:
# Dataset
inputsize = 8
nr= 12
data= np.random.random(size=(nr,inputsize,2))
ds = zip(data,data,data)
for i in ds:
    print(i[0].shape)



(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)
(8, 2)


In [ ]:
class CModel(tf.keras.Model):
 
    def train_step(self, x):
        print(x)
        return {m.name: m.result() for m in self.metrics}


class Base:
    """ The basic model oshea resnet """
    def __init__(self,input_shape=8,output_shape=6,name='Base') -> None:
        ## Inputs
        self.input_layer = tf.keras.Input(shape=(input_shape,2))
        self.reshaper = tf.keras.layers.Reshape((1,input_shape,2))
        self.featEx = FeatureExtraction(nr_layers=4)
        self.output= Exit(output_shape)
    
    def build(self):
        x = self.reshaper(self.input_layer)
        x= self.featEx(x)
        out = self.output(x)
        return out, 'Full_model'
         
    def get_model(self,m=-1):
        out,post = self.build(m)
        return CModel(self.input_layer,out,name=post)

class FeatureExtraction(CModel):
    def __init__(self, nr_layers=7, filters=32,kernel_size=(1,3)):
        super(FeatureExtraction, self).__init__()
        self.ccns = [tf.keras.layers.Conv2D(filters=filters,kernel_size=kernel_size) for _ in range(nr_layers) ]
        self.max_pool = tf.keras.layers.MaxPool2D((1,2))

    def call(self, x, training=False):
        for stack in self.cnns:
            x = stack(x,training=training)
            x = self.max_pool(x)
        return x

class FeatureExtraction(CModel):
    def __init__(self, nr_layers=7, filters=32,kernel_size=(1,3)):
        super(FeatureExtraction, self).__init__()
        self.ccns = [tf.keras.layers.Conv2D(filters=filters,kernel_size=kernel_size) for _ in range(nr_layers) ]
        self.max_pool = tf.keras.layers.MaxPool2D((1,2))

    def call(self, x, training=False):
        for stack in self.cnns:
            x = stack(x,training=training)
            x = self.max_pool(x)
        return x

class Exit(CModel):
    """ Decision layer, this are multiple fully conected stacks + classefing stack """
    def __init__(self,output_shape,N_per_dense = (128,128) ):
        super(Exit, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense_layers = [tf.keras.layers.Dense(nn,activation=tf.keras.activations.selu) for nn in N_per_dense ]
        self.output_layer = tf.keras.layers.Dense(output_shape,activation=tf.keras.activations.softmax, name="Exit")
        self.dropout= tf.keras.layers.Dropout(0.1)

    def call(self, input_tensor, training=False):
        x = self.flatten(input_tensor)
        for layer in self.dense_layers:
            x = layer(x,training=training)
            x= self.dropout(x,training=training)
        out = self.output_layer(x,training=training)
        return out
    
    def get_output(self):
        return self.output_layer

: 

In [ ]:
cnns=4
connected=(128,128)
output_shape = 6

input_shape= inputsize
input_layer = tf.keras.Input(shape=(input_shape,2))
x = tf.keras.layers.Reshape((1,input_shape,2))(input_layer)
for _ in range(cnns):
    x =  tf.keras.layers.Conv2D(filters=32,kernel_size=(1,3))(x)
    x = tf.keras.layers.MaxPool2D((1,2))(x)

for nn in connected:
    x = tf.keras.layers.Dense(nn,activation=tf.keras.activations.selu)(x)
    x= tf.keras.layers.Dropout(0.1)(x)
out = tf.keras.layers.Dense(output_shape,activation=tf.keras.activations.softmax, name="Exit")

model = tf.keras.Model(input_layer,out)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(ds,epochs=10)